# Clase Nodo

### Permite la creación de cada nodo del grafo y, por ende la creación de los árboles.


In [1]:
class Node:
    
    """
    
    Clase para crear nodos -y por lo tanto árboles - y guardar la información necesaria en cada caso
    
    """
    
    def __init__(self, label, pos, origen = 'Otro', parent= None, dep='ROOT'):
        self.label = label
        self.parent = parent
        self.dep = dep
        self.pos = pos
        self.children = []
        self.origen = origen
        self.oracion = ''
        

    def add_child(self, Node):
        self.children.append(Node)
        return self
    
    def add_children(self, list_of_childs):
        for new_child in list_of_childs:
            self.add_child(new_child)  
        return self
            
    def change_parent(self, new_parent):
        
        self.parent = new_parent
        
    def tree_to_matrix(self):
        
        nodes = DFS(self)
               
        node_list = []
        adj_mtrx = []
        
        for i, nodei in enumerate(nodes):
            mat_i_row = []
            
            for j, nodej in enumerate(nodes):
                if nodej.parent == nodei:
                    mat_i_row.append(j)
            
            adj_mtrx.append(mat_i_row)
            node_list.append(nodei.dep)
            
        return node_list, adj_mtrx
    

    def __repr__(self):
        
        """
            Aunque puede hacerce mejor, esta función está diseñada para mantener la notación de (Paaßen,2018)
        """
        
        ret = self.label
        if len(self.children) != 0:
            ret += '('
        for child in self.children:
            ret += child.__repr__() +','
        if len(self.children) != 0:
            ret = ret[:-1]
            ret += ')'
        return ret
    
    def __len__(self):
        
        count = 1
        
        for child in self.children:
            
            count += len(child)
        
        return count
    



#-----------------#
#     Ejemplo     #
#-----------------#

T1 = Node('está', 'Aux')
T1.add_child(Node('Esto', 'PRON', 'está', 'nsubj'))
T1.add_child(Node('alquilar', 'VERB', 'está', 'ccomp'))
T1.children[1].add_child(Node('como', 'SCONJ', 'alquilar', 'mark'))
T1.children[1].add_child(Node('para', 'ADP', 'alquilar', 'mark'))
T1.children[1].add_child(Node('balcón', 'NOUN', 'alquilar', 'obj'))


print('Este es un arbol de ejemplo:')
print(T1.__repr__())


###################################################################################################
#                   OTRAS FUNCIONES ÚTILES
###################################################################################################

def DFS(tree):
    
    """
        Función para obtener el pre-order de un árbol (o bosque)
    """
    
    pre_order = []
    
    pre_order.append(tree)
    
    for child in tree.children:
        
        pre_order = pre_order + DFS(child)
        
    return pre_order

print('Este es el resultado de DFS en ese arbol:')
print(DFS(T1))

#------------------------------------------------------------------------------------

T_prueba = Node('a',0)
T_prueba.add_child(Node('b',0))
T_prueba.children[0].add_child(Node('c',0))
T_prueba.children[0].add_child(Node('d',0))
T_prueba.add_child(Node('e',0))


print(T_prueba)


f = [T_prueba]  # Se define un bosque con un solo árbol para que funcione el algoritmo

def Subforest(forest, i, j, k):
    
    Y = []
    
    for r in forest:
        k += 1
        if k > j:
            return (Y,k)
        elif k >= i:
            (Y1 , k) = Subforest(r.children,i,j,k)
            Y.append(Node(r.label, r.pos,  r.parent, r.dep).add_children(Y1))
        else:
            (Y1 , k) = Subforest(r.children,i,j,k)
            Y = Y1    
    
    return (Y,k)

print(Subforest(f,2,4,0))


#---------------------------------------------------------

def rl(tree, i):
    f = [tree]
    return  i + len(Subforest(f,i,len(tree),0)[0][0]) -1
    
print(rl(T_prueba,1))


#--------------------------------------------------------

def Cost_Function(n1,n2):
    
    return 1

Este es un arbol de ejemplo:
está(Esto,alquilar(como,para,balcón))
Este es el resultado de DFS en ese arbol:
[está(Esto,alquilar(como,para,balcón)), Esto, alquilar(como,para,balcón), como, para, balcón]
a(b(c,d),e)
([b(c,d)], 5)
5


# Generar el Corpus etiquetado

### Utilizando el *framework* Universal Dependencies.


In [4]:
from glob import glob
import os.path
from pathlib import Path


In [3]:
#Importar la librería Spacy y el modelo de etiquetado para la lengua española

import spacy 
nlp = spacy.load('es_core_news_md')


# Definir el texto a etiquetar

# make an empty string to hold the data
text = ''

# # for each text file in the data folder,
for filepath in glob("Corpus/puntuacion_corregida/*.txt"):   # Ruta del Corpus completo sin modificar : "Corpus/*.txt"
    with open(filepath, 'r') as file:
        text += file.read()


In [7]:
bosque = [] #Variable en la que se guardan todos los árboles a comparar. 

ciudad = 'Otro'

#Etiquetar el texto
frase_etiquetada = ''
for frase in text.splitlines():
    
    
    
    if ('[' in frase) and (']' in frase):
        pass
    elif '~*' in frase:
        ciudad = frase[3:-3]
    else:
        frase_etiquetada = nlp(frase.strip())
        
        try:
            options = {"compact": True, "bg": "#09a3d5",
                   "color": "black", "font": "Source Sans Pro"}
            # spacy.displacy.render(frase2, style="dep", options=options, page=True) 
            svg = spacy.displacy.render(frase_etiquetada, style="dep", options=options, jupyter=False)
            file_name = '-'.join([w.text for w in frase_etiquetada if not w.is_punct]) + ".svg"
            output_path = Path("./grafos/" + file_name)
            output_path.open("w", encoding="utf-8").write(svg)
        except:
            print("la frase:\n {}\n no pudo imprimirse como grafo".format(frase_etiquetada))
    
    # Crear un nodo por cada token, con la información necesaria.
    
    lista_nodos = []
    
    for token in frase_etiquetada:
        
        if str(token.pos_) == 'SPACE' or str(token.pos_) == 'PUNCT':
            pass
        else:
            lista_nodos.append(Node(str(token), str(token.pos_), ciudad, str(token.head), str(token.dep_)))
        
    # Crear las dependencias entre los nodos
        
    for i in lista_nodos:
        for j in lista_nodos:
            
            if j.parent == i.label and j.dep != 'ROOT':
                
                i.add_child(j)
                j.change_parent(i)
                
        if i.dep == 'ROOT':
            
            i.oracion = frase            
            bosque.append(i)

la frase:
 A veces quiero vivir en un mundo mágico; sin que haya nada trágico, sin que haya nada plástico, donde lo único que se nos pase rápido sea la tristeza y  la belleza, sea un cuore fantástico; donde el látigo que la justicia vende  sea un ejemplo consecuente y no haya gente indiferente; estar permanente donde el dolor se olvida y el error sea el señor mentor, que hacer mejor con vida.
 no pudo imprimirse como grafo


In [2]:
! pip install cairosvg

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.7 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [ ]:
import cairosvg

for i, filepath in enumerate(glob("grafos/*.svg")):   # Ruta del Corpus completo sin modificar : "Corpus/*.txt"
    
    # Convert SVG file to PNG file
    cairosvg.svg2png(url=filepath, write_to="grafos/{}.png".format(i))

In [ ]:
print(len(bosque))

In [ ]:
for i,arbol1 in enumerate(bosque):
    for j,arbol2 in enumerate(bosque):
        if i!=j and (arbol1.__repr__() == arbol2.__repr__()):
            bosque.remove(arbol2)

In [ ]:
print(len(bosque))

### Uso de la implementación del algoritmo de Zhang & Shasha de Paaßen

https://pypi.org/project/edist/#description

In [ ]:
from edist.ted import standard_ted

standard_ted(bosque[0].tree_to_matrix(), bosque[3].tree_to_matrix())  

In [ ]:
from edist.ted import standard_ted
import numpy as np

dims = (len(bosque),len(bosque))

dist_mat = np.zeros(dims)

for i, arbol1 in enumerate(bosque):
    for j, arbol2 in enumerate(bosque):        
        dist_mat[i][j] = standard_ted(arbol1.tree_to_matrix(),arbol2.tree_to_matrix())

   
print(dist_mat)

In [ ]:
# import sklearn
from sklearn_extra.cluster import KMedoids
import matplotlib.pyplot as plt

#Probar cual número de clusters es mejor.

inertia = []

for n in range(1,200): 
    kmedoids = KMedoids(n_clusters=n, random_state=0).fit(dist_mat)
    inertia.append(kmedoids.inertia_)

In [ ]:
import seaborn as sns

sns.set_style("darkgrid", {"axes.facecolor": ".9"})

# plt.scatter(range(1,200),inertia)
ax = sns.scatterplot(data=None, x =range(1,200), y =inertia )
ax.set(xlabel='Número de clústers', ylabel='Inercia')

# print(inertia[50],inertia[198])

In [ ]:
kmedoids = KMedoids(n_clusters=25, random_state=0).fit(dist_mat)
print(kmedoids.inertia_)
print(kmedoids.labels_)

## Análisis de los resultados 

In [ ]:
groups = []

for i in range(max(kmedoids.labels_)+1):
    group = []
    for j, index in enumerate(kmedoids.labels_):
        if i == index:
            group.append(bosque[j])
    groups.append(group)
    
print(max(kmedoids.labels_))   

In [ ]:
origenes_totales = []

for i, group in enumerate(groups):
    for tree in group:
        origenes_totales.append(tree.origen)
    
print((origenes_totales.count('Bogotá')/len(bosque))*100)
print((origenes_totales.count('Medellín')/len(bosque))*100)
print((origenes_totales.count('Otro')/len(bosque))*100)

In [ ]:

for i, group in enumerate(groups):
    origenes = []
    for tree in group:
        origenes.append(tree.origen)
    
    bog = (origenes.count('Bogotá')/len(group))*100
    med = (origenes.count('Medellín')/len(group))*100
    otr = (origenes.count('Otro')/len(group))*100
        
    print("""_________________________________________________
GRUPO {}:                           ({} elementos)
  
  ⋅ Porcentaje Bogotá         {}%
  ⋅ Porcentaje Medellín       {}%
  ⋅ Porcentaje Otro           {}%""".format(i,len(group),bog,med,otr))
        

In [ ]:

import numpy as np

for i, group in enumerate(groups):
    largos = []
    for tree in group:
        largos.append(len(tree))
    
    len_i = (sum(largos)/len(largos))
    len_std = np.std(largos)
        
    print("""{} & {:.2f} & {:.2f} \\""".format(i,len_i,len_std))

In [ ]:
for i in groups[3]:
    print(len(i))#.oracion)

In [ ]:
"""                       ,-=-.       ______     _
                         /  +  \     />----->  _|1|_
                         | ~~~ |    // -/- /  |_ H _|
                         |R.I.P|   //  /  /     |S|
                    \vV,,|_____|V,//_____/VvV,v,|_|/,,vhjwv/

  ▄████▄ ▓█████ ███▄ ▄███▓█████ ███▄    █▄▄▄█████▓█████ ██▀███  ██▓▒█████  
 ▒██▀ ▀█ ▓█   ▀▓██▒▀█▀ ██▓█   ▀ ██ ▀█   █▓  ██▒ ▓▓█   ▀▓██ ▒ ██▓██▒██▒  ██▒
 ▒▓█    ▄▒███  ▓██    ▓██▒███  ▓██  ▀█ ██▒ ▓██░ ▒▒███  ▓██ ░▄█ ▒██▒██░  ██▒
 ▒▓▓▄ ▄██▒▓█  ▄▒██    ▒██▒▓█  ▄▓██▒  ▐▌██░ ▓██▓ ░▒▓█  ▄▒██▀▀█▄ ░██▒██   ██░
 ▒ ▓███▀ ░▒████▒██▒   ░██░▒████▒██░   ▓██░ ▒██▒ ░░▒████░██▓ ▒██░██░ ████▓▒░
 ░ ░▒ ▒  ░░ ▒░ ░ ▒░   ░  ░░ ▒░ ░ ▒░   ▒ ▒  ▒ ░░  ░░ ▒░ ░ ▒▓ ░▒▓░▓ ░ ▒░▒░▒░ 
   ░  ▒   ░ ░  ░  ░      ░░ ░▓█████▄▓█████   ░    ░ ░  ░ ░▒ ░ ▒░▒ ░ ░ ▒ ▒░ 
 ░          ░  ░      ░     ░▒██▀ ██▓█   ▀ ░        ░    ░░   ░ ▒ ░ ░ ░ ▒  
 ░ ░        ░  ░      ░     ░░██   █▒███            ░  ░  ░     ░     ░ ░  
 ░                           ░▓█▄   ▒▓█  ▄                                              
                             ░▒████▓░▒████▒
                              ▒▒▓  ▒░░ ▒░ ░
     ██████    ██ ███▄    █ ▄████▄  ██▓▒█████  ███▄    █▓█████  ██████ 
   ▓██   ▒██  ▓██▒██ ▀█   █▒██▀ ▀█ ▓██▒██▒  ██▒██ ▀█   █▓█   ▀▒██    ▒ 
   ▒████ ▓██  ▒██▓██  ▀█ ██▒▓█    ▄▒██▒██░  ██▓██  ▀█ ██▒███  ░ ▓██▄   
   ░▓█▒  ▓▓█  ░██▓██▒  ▐▌██▒▓▓▄ ▄██░██▒██   ██▓██▒  ▐▌██▒▓█  ▄  ▒   ██▒
   ░▒█░  ▒▒█████▓▒██░   ▓██▒ ▓███▀ ░██░ ████▓▒▒██░   ▓██░▒████▒██████▒▒
    ▒ ░  ░▒▓▒ ▒ ▒░ ▒░   ▒ ▒░ ░▒ ▒  ░▓ ░ ▒░▒░▒░░ ▒░   ▒ ▒░░ ▒░ ▒ ▒▓▒ ▒ ░
    ░    ░░▒░ ░ ░░ ░░   ░ ▒░ ░  ▒   ▒ ░ ░ ▒ ▒░░ ░░   ░ ▒░░ ░  ░ ░▒  ░ ░
    ░ ░   ░░░ ░ ░   ░   ░ ░░        ▒ ░ ░ ░ ▒    ░   ░ ░   ░  ░  ░  ░  
            ░             ░░ ░      ░     ░ ░          ░   ░  ░     ░  
                           ░                                        
"""



# FUNCIONES OLVIDADAS O PARA CAMBIAR/GUARDAR



# Función de representación antigua de los árboles:

#     def __repr__(self, level=0):
#         ret = '\t' * level + self.label + '\n'
#         for child in self.children:
#             ret += child.__repr__(level+1)
#         return ret





# Intento de implementar Zhang & Shasha               

# import numpy as np

# T2 = Node('f',0).add_child(Node('g',0))

# print(T_prueba)
# print(T2)

# def Tree_edit_distance(X,Y,c=None):
    
#     m = len(X)
#     n = len(Y)   
 
#     d = np.zeros((m,n))
#     D = np.zeros((m+1,n+1))
#     print(D)
      
#     for k in reversed(range(m)):
#         for l in reversed(range(n)):
#             D[rl(X,k)+1][rl(Y,l)+1] = 0
#             for i in reversed(range(k,rl(X,k))):
#                 D[i][rl(Y,l)+1] = D[i+1][rl(Y,l)+1] + 1 #Cost_Function(X[i],Node('Empty', None)) 
#                 print(D)
#             for j in reversed(range(l,rl(Y,l))):
#                 D[rl(X,k)+1][j] = D[rl(X,k)+1][j+1] + 1 #Cost_Function(Node('Empty', None),Y[j]) 
#                 print(D)
#             for i in reversed(range(k,rl(X,k))):
#                 for j in reversed(range(l,rl(Y,l))):
#                     if rl(X,i) == rl(X,k) and rl(Y,j) == rl(Y,l):
#                         D[i][j] = min(D[i+1][j] + 1, # Cost_Function(X[i],Node('Empty', None)) ,
#                                       D[i][j+1] + 1, #Cost_Function(Node('Empty', None),Y[j]) ,
#                                       D[rl(X,i)+1][rl(Y,j)+1] + 1) #Cost_Function(X[i],Y[j]))
#                         print(D)
#                         d[i][j] = D[i][j]
#                     else:
#                         D[i][j] = min(D[i+1][j] + 1, #Cost_Function(X[i],Node('Empty', None)) ,
#                                       D[i][j+1] + 1, # Cost_Function(Node('Empty', None),Y[j]) ,
#                                       D[rl(X,i)+1][rl(Y,j)+1] + d[i][j]) 
#                         print(D)
            
#     return d[0][0]
    
# Tree_edit_distance(T_prueba,T2)

